<a href="https://colab.research.google.com/github/makingthefuturehappy/bonus/blob/main/weekend_bonuses%20v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<H1>Bonus</H1>

Drivers' segments description
https://docs.google.com/spreadsheets/d/11KIaZaywoBq3MymrCd8dmWJdBhFO989SNsrrSKYiB3Y/edit#gid=0<br><br>
2 steps:<br>
1st step: cohorts and features calc<br>
2nd step: business case calc<br>

**Настраиваем переменные расчета**

In [ ]:
path = "SLP12.csv"
cohorts = 5           # quantity of cohorts
commission = 0.0114    # indriver commission
win_rate = 0.15       # percent of drivers who moves to the upper cohort
period_duration = 0.5 # period to forecast prospective rides from the total period

## Загружаем данные и анализируем кол-во поездок для каждого водителя

In [ ]:
import pandas as pd
import numpy as np
import math
from datetime import datetime
import hashlib
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# percentile level calc
percentile_level = {}
percentile = 0
for i in range (1, cohorts):
    percentile = 100 / cohorts + percentile
    percentile_level[i] = int(percentile)

<h3>Data upload and pre-calc</h3>

In [ ]:
df = pd.read_csv(path)
df['gmv'] = pd.to_numeric(df['gmv'], errors='coerce')
dates = list(df.columns)[2:]
print("total drivers:", df.shape[0])
df.sample(n=5)

total drivers: 5926


,id,gmv,02.01.23,03.01.23,04.01.23,05.01.23,06.01.23,07.01.23,08.01.23,09.01.23,...,20.01.23,21.01.23,22.01.23,23.01.23,24.01.23,25.01.23,26.01.23,27.01.23,28.01.23,29.01.23
4653,129791310,417,5,2,7,4,4,3,1,3,...,10,0,4,3,7,5,5,9,0,4
5317,16087374,1893,16,13,22,19,20,22,25,19,...,19,25,20,21,22,22,20,19,27,27
1988,157080742,36,0,0,0,0,0,0,0,0,...,0,0,0,0,5,2,0,0,0,0
4247,156919560,141,0,0,0,2,3,5,0,0,...,0,5,3,0,0,4,0,0,0,13
4576,36876325,33,0,0,0,0,0,0,0,0,...,0,10,0,0,0,0,0,0,0,0


<h3>retention & avr. check calc</h3>

In [ ]:
# date split by weekends and workdays 
fridays = []
saturdays = []
sundays = []
workdays = []

dates = list(filter(lambda x: "." in x, df.columns))
for weekday in dates:
    daynum = datetime.strptime(weekday, '%d.%m.%y').isoweekday()
    if daynum == 5: fridays.append(weekday)
    elif daynum == 6: saturdays.append(weekday)
    elif daynum == 7: sundays.append(weekday)
    else: workdays.append(weekday)

In [ ]:
# split the period by 2 equal "dates" parts 
if len(dates)%2 == 0:
    index_2nd = len(dates)//2
    index_1st = 0
else:
    index_2nd = len(dates)//2+1
    index_1st = 1
    
first_half = dates[index_1st : index_2nd]
second_half = dates[index_2nd : len(dates)]
total_days = len(first_half) + len(second_half)


# retention & avr_check cals
df['first_half'] = df[first_half].sum(axis=1)
df['second_half'] = df[second_half].sum(axis=1)

def retention_calc(first_half, second_half):
    try:
        retention = second_half / first_half
    except:
        retention = 0
    retention = min (1, retention)
    return retention

df['retention'] = df.apply(lambda x: retention_calc(x['first_half'], x['second_half']), axis =  1)
df['total_rides'] = df['first_half'] + df['second_half']
df['avr_check'] = df['gmv'] / df['total_rides']
df['WK_rides'] = df[fridays + saturdays + sundays].sum(axis=1) # weekends rides
df['WD_rides'] = df[workdays].sum(axis=1) # weekends rides


key_fields = ['id', 'gmv','first_half','second_half','total_rides', 'WD_rides','WK_rides', 'avr_check', 'retention'] 
 
df = df[np.isfinite(df).all(1)] # infinity values drop (comes from a division by zero)
df = df.round({'gmv': 0, 'retention': 2, 'avr_check': 1})

## Результат расчета кол-ва поездок


**id** - id водителя<br>
**gmv** - gmv водителя<br>
**first_half** - количество поездок в первую половину периода<br>
**second_half** - количество поездок в первую половину периода<br>
**total_rides** - всего поездок за период
**WD_rides** - количество поездок в WorkDays<br>
**WK_rides** - количество поездок в WeeKends<br>
**avr_check** - средний чек<br>



In [ ]:
print('total drivers:', df.shape[0])
print("\nexample of random drivers'id from the dataset")
df[key_fields].sample(n=5)

total drivers: 5926

example of random drivers'id from the dataset


,id,gmv,first_half,second_half,total_rides,WD_rides,WK_rides,avr_check,retention
3319,16999766,827,147,105,252,159,93,3.3,0.71
3607,158182955,39,0,12,12,6,6,3.2,1.00
4295,119613940,36,11,0,11,0,11,3.3,0.00
4721,17920666,384,69,48,117,29,88,3.3,0.70
8,158042485,469,8,135,143,78,65,3.3,1.00


## Расчитываем размеры когорт

In [ ]:
# exclude IDs which have zero rides during the first half of the period
df_activer_drivers = df.loc[df['first_half'] != 0]
    
all_cohorts = {}
range_start = 0

for percentile in percentile_level.keys():
    
    max_value = np.percentile(np.array(df_activer_drivers.total_rides.tolist()), percentile_level[percentile]) # percentile value
    all_cohorts[percentile] = range(int(range_start), int(max_value))
    range_start = max_value

last_percentile = int(max(list(percentile_level.keys())))+1
all_cohorts[last_percentile] = range(int(max_value),
                                     int(1e8)) # set a large number to keep the cohort range            

# cohort distribution for all IDs
def cohort_check(x, all_cohorts):
    for cohort in all_cohorts.keys():
        if x in all_cohorts[cohort]:
            return cohort
        else:
            pass      
df['cohort'] = df.apply(lambda x: cohort_check(x['total_rides'], all_cohorts), axis =  1)

In [ ]:
# shuffle data within cohorts
def shuffle(df,      # dataframe to shuffle data 
            cohorts): # cohorts to shuffle data within
    shuffle_df = pd.DataFrame()
    for cohort in set(cohorts):
        shuffle_df = shuffle_df.append(df[df.cohort == cohort].sample(frac=1))
    return shuffle_df

df = shuffle(df, df['cohort'].tolist())
df.reset_index(drop=True, inplace = True)
df.sample(n=5)

,id,gmv,02.01.23,03.01.23,04.01.23,05.01.23,06.01.23,07.01.23,08.01.23,09.01.23,...,28.01.23,29.01.23,first_half,second_half,retention,total_rides,avr_check,WK_rides,WD_rides,cohort
2910,52428076,246,2,1,4,4,3,7,6,1,...,7,6,41,34,0.83,75,3.3,51,24,3
1970,19215836,79,0,0,0,0,0,0,0,0,...,3,2,2,22,1.00,24,3.3,20,4,2
3114,34293599,253,0,0,0,8,10,9,6,4,...,5,0,48,29,0.60,77,3.3,48,29,3
4501,49140712,485,0,0,2,0,2,2,8,2,...,13,4,44,104,1.00,148,3.3,82,66,4
5193,66384659,941,9,9,9,7,12,16,9,7,...,14,11,118,169,1.00,287,3.3,126,161,5


<H3>cohorts features calc</H3>

In [ ]:
avr_check = {}
for cohort in range(1, cohorts+1):
    avr_check[cohort] = df['avr_check'].loc[df['cohort'] == cohort].mean()

avr_rides = {}
for cohort in range(1, cohorts+1):
    avr_rides[cohort] = df['total_rides'].loc[df['cohort'] == cohort].mean()
    
avr_wd_rides = {}
for cohort in range(1, cohorts+1):
    avr_wd_rides[cohort] = df['WD_rides'].loc[df['cohort'] == cohort].mean()
    
avr_wk_rides = {}
for cohort in range(1, cohorts+1):
    avr_wk_rides[cohort] = df['WK_rides'].loc[df['cohort'] == cohort].mean()

drivers = {}
for cohort in range(1, cohorts+1):
    drivers[cohort] = df['id'].loc[df['cohort'] == cohort].count()

avr_retention = {}
for cohort in range(1, cohorts+1):
    avr_retention[cohort] = df['retention'].loc[df['cohort'] == cohort].mean()

cohort_df = pd.DataFrame(list(avr_check.items()),
                   columns=['cohort', 'avr_check'])
cohort_df['drivers'] = list(drivers.values())
cohort_df['avr_rides'] = list(avr_rides.values())
cohort_df['WK_all_rides'] = list(avr_wk_rides.values()) # quantity of rides on weekends 
cohort_df['WD_all_rides'] = list(avr_wd_rides.values()) # quantity of rides on workdays 


# 'WK_avr_rides' - # avr. rides on a particular weekend
weekends_days = np.mean([len(fridays), len(saturdays), len(sundays)])
cohort_df['WK_avr_rides'] = cohort_df['WK_all_rides'] / weekends_days

# 'WD_avr_rides' - # avr. rides on a particular weekend
workdays_days = len(workdays)
cohort_df['WD_avr_rides'] = cohort_df['WD_all_rides'] / workdays_days

# retention calc
cohort_df = pd.merge(df.groupby('cohort')['retention'].mean(),
                     cohort_df,
                     on="cohort")

cohort_df['avr_rev_total'] = (cohort_df['avr_check'] * cohort_df['avr_rides'] * commission).astype(int)
cohort_df = cohort_df.round({'retention': 2,
                             'avr_check': 1,
                             'avr_rides': 1,
                             'WK_all_rides': 1,
                             'WD_all_rides': 1,
                             'WK_avr_rides': 1,
                             'WD_avr_rides': 1,
                            })

## Описание когорт 

In [ ]:
print('\ncohorts and range of rides:')
all_cohorts


cohorts and range of rides:


{1: range(0, 22),
 2: range(22, 62),
 3: range(62, 119),
 4: range(119, 209),
 5: range(209, 100000000)}

In [ ]:
print("описание когорт:")
cohort_df

описание когорт:


,cohort,retention,avr_check,drivers,avr_rides,WK_all_rides,WD_all_rides,WK_avr_rides,WD_avr_rides,avr_rev_total
0,1,0.57,3.3,1515,8.7,4.8,3.9,1.2,0.2,0
1,2,0.72,3.3,1228,39.2,21.6,17.5,5.4,1.1,1
2,3,0.82,3.3,1107,89.3,46.9,42.4,11.7,2.7,3
3,4,0.86,3.3,1043,158.9,78.1,80.9,19.5,5.1,5
4,5,0.90,3.3,1033,311.5,140.2,171.3,35.1,10.7,11


<h3>Target Rides and Incentive</h3>

In [ ]:
# bonus to be given as an inventive by groups, % from average check of the cohort  
# 1 equals 100% from the GMV of the target rides
# example: gmv = $10, 0.35 = $3.5 bonus will be given 

bonuses = {           
    1: 0,
    2: 1,
    3: 1,
    4: 1,
    5: 0}

## Расчитываем цели и вознаграждения

In [ ]:
# cohort_rides - maximum rides out of workdays & weekends rides
#cohort_df['cohort_rides'] = cohort_df[['WK_avr_rides', 'WD_avr_rides']].max(axis=1)

# rounding up all rides
cohort_df['cohort_rides'] = cohort_df['WK_avr_rides'].apply(lambda x: math.ceil(x))

# calc the target rides
new_rides = {}
for cohort in range(1, cohorts):
    new_rides[cohort] = cohort_df.loc[cohort,'cohort_rides'] - cohort_df.loc[cohort-1,'cohort_rides']

new_rides[cohorts] = 0 # set zero for the last upper cohort as there is no insentive

# 'WK_rides_extra' - Weekend Extra Rides to be done
cohort_df['WK_rides_extra'] = cohort_df['cohort'].map(new_rides) + 1
# add 1 additional ride to overachieve cohort border 
    
# 'rides_tbd' - all weekends rides to get the bonus
cohort_df['rides_tbd'] = cohort_df['cohort_rides'] + cohort_df['WK_rides_extra']
cohort_df.loc[cohorts-1,'rides_tbd'] = 0 # the target for the senior cohort shall be zero

# incentive for drivers
cohort_df['bonus, %'] = cohort_df['cohort'].map(bonuses)
cohort_df['driver_bonus'] = cohort_df['WK_rides_extra'] * cohort_df['avr_check'] * cohort_df['bonus, %']
cohort_df = cohort_df.round({'driver_bonus': 0})

## Bonus adjustments (if required)
 

pre-calc bonuses based on % of the Average Check<br>

In [ ]:
cohort_df[['cohort', 'avr_check', 'bonus, %', 'driver_bonus']]

,cohort,avr_check,"bonus, %",driver_bonus
0,1,3.3,0,0.0
1,2,3.3,1,23.0
2,3,3.3,1,30.0
3,4,3.3,1,56.0
4,5,3.3,0,0.0


below is a correction of the pre-calc bonuses for each cohort (absolut value in USD)

In [ ]:
driver_bonus = {           
    1: 5.0,
    2: 4.0,
    3: 8.0,
    4: 17.0,
    5: 0.0}

cohort_df['driver_bonus'] = cohort_df['cohort'].map(driver_bonus)

## Rides adjustment (if required)

below is a correction of the pre-calc rides for each cohort.<br>
Figures are <u>increment</u> to "rides_tbd".<br>
Adjustment can be both negative and positive.



In [ ]:
# figures to be changed:
rides_adj = {1:0,
             2:3,
             3:2,
             4:3,
             5:0
             }

In [ ]:
rides_current = cohort_df['rides_tbd']

rides_adj = rides_adj.values()
rides_current = list(map(sum, zip(rides_current, rides_adj)))
rides_current = dict(zip([i for i in range(1, cohort+2)], rides_current))

cohort_df['rides_tbd'] = cohort_df['cohort'].map(rides_current)
print("new rides:")
for key, value in rides_current.items():
  print(key,":", value)

new rides:
1 : 7
2 : 16
3 : 23
4 : 40
5 : 0


## Результат расчетов

**WD_avr_rides**    - среднее кол-во поездок в workdays<br> 
**WK_avr_rides**    - среднее кол-во поездок в weekends<br>
**cohort_rides**    - среднее кол-во поездок в когорте<br>
**WK_rides_extra**  - на сколько больше надо сделать поездок в выходные<br>
**rides_tbd**       - сколько в итоге надо сделать поездок в выходые всего<br>
**driver_bonus**    - сколько бонусов предлагаем водителю за выполнение цели<br>
**avr_check**       - средний чек водителя (для сравнения)<br>

In [ ]:
cohort_df[['cohort',
           'WD_avr_rides',
           'WK_avr_rides',
           'cohort_rides',
           'WK_rides_extra',
           'rides_tbd',
           'driver_bonus',
           'avr_check']]

,cohort,WD_avr_rides,WK_avr_rides,cohort_rides,WK_rides_extra,rides_tbd,driver_bonus,avr_check
0,1,0.2,1.2,2,5,7,5.0,3.3
1,2,1.1,5.4,6,7,16,4.0,3.3
2,3,2.7,11.7,12,9,23,8.0,3.3
3,4,5.1,19.5,20,17,40,17.0,3.3
4,5,10.7,35.1,36,1,0,0.0,3.3


## Расчитываем окупаемость

<h3>Instant revenue and costs</h3>

In [ ]:
# instant revenue
cohort_df['win_rate'] = win_rate

# costs
cohort_df['costs'] = cohort_df['driver_bonus'] * \
    cohort_df['drivers'] * win_rate

# extra revenue for the company
cohort_df['ID_extra_rev'] = cohort_df['WK_rides_extra'] \
    * cohort_df['avr_check'] \
    * cohort_df['drivers'] \
    * win_rate \
    * commission

instant_rev = cohort_df['ID_extra_rev'].sum()
instant_cost = cohort_df['costs'].sum()

print("instant revenue:", int(instant_rev))
print("instant costs  :", int(instant_cost))
print("instant balance:", int(instant_rev - instant_cost))

instant revenue: 253
instant costs  : 5861
instant balance: -5607


<h2>Perspective revenue projection</h2>

<h3>Target is not fullfilled</h3>

In [ ]:
# target IS NOT fulfilled
# 'P: not FF: rev' - Perspective Not Fulfilled: Revenue

cohort_df['P: not FF: rev'] = cohort_df['avr_rides'] \
                                * cohort_df['avr_check'] \
                                * cohort_df['retention'] \
                                * period_duration \
                                * commission
cohort_df.loc[cohorts-1,'P: not FF: rev'] = 0 #keep the last cohort zero as no an insentive for the cohort

<h3>Target is fullfilled</h3>

In [ ]:

# target IS fulfilled
FF_rides = {}
for i in range (1, cohorts):
    FF_rides[i] = avr_rides[i+1] * period_duration
FF_rides[cohorts] = 0
cohort_df['P: FF: rides'] = list(FF_rides.values())

FF_gmv = {}
for i in range (1, cohorts):
    FF_gmv[i] = int(FF_rides[i] * avr_check[i+1])
FF_gmv[cohorts] = 0
cohort_df['P: FF: gmv'] = list(FF_gmv.values())

FF_rev = {}
for i in range (1, cohorts):
    FF_rev[i] = int(FF_gmv[i] * commission * avr_retention[i+1])
FF_rev[cohorts] = 0
cohort_df['P: FF: rev'] = list(FF_rev.values())

cohort_df['P: rev_up'] = (cohort_df['P: FF: rev'] - cohort_df['P: not FF: rev']) \
                                * win_rate \
                                * cohort_df['drivers']

persp_rev_up = (cohort_df['P: rev_up'].sum())
revenue_total = cohort_df['ID_extra_rev'].sum() + persp_rev_up

## Результат расчета

In [ ]:
print('instant revenue      :', int(instant_rev))
print('perspective revenue  :', int(persp_rev_up))
print('revenue total        :', int(revenue_total))
print('costs:               :', int(instant_cost))
print('profit:              :', int(revenue_total - instant_cost))

instant revenue      : 253
perspective revenue  : 548
revenue total        : 802
costs:               : 5861
profit:              : -5059


## Распределяем водителей по группам для тестов разного размера бонусов

распределяем долю бонусов от расчитанных, которые мы отдаем до тестов<br>
**control** - не даем бонусы<br>
**остальные** - 1: отдаем 100% расчитанных бонусов, 0.5% отдаем 50% и т.д.

In [ ]:
bonus_coeff = {
    "0": 0,       # control group
    "1": 0.8,       # 100% of target bonuses will be given to drivers
    "2": 0.8,     # 
    "3": 1}     # 

In [ ]:
def get_checksum(driver_df: pd.DataFrame, salt = "some salt") -> pd.DataFrame:
    # random distribution drivers per groups 
    byte_arr = driver_df['id'].astype(str) + salt
    byte_arr = byte_arr.str.encode(encoding='UTF-8')
    driver_df['md5'] = byte_arr.apply(lambda a: int(hashlib.md5(a).hexdigest(), 16))
    return df

def set_subgroups(df: pd.DataFrame, bonus_coeff: dict) -> pd.DataFrame:
    if not 'md5' in df.columns:
        raise KeyError('No md5 column')
    groups = len(bonus_coeff)
    for cohort in range(1, cohorts+1):
      df.loc[df['cohort'] == cohort, 'group'] = (df['md5'] % groups).astype('string')
    df['bonus_coeff'] = df['group'].map(bonus_coeff)
    return df

df = get_checksum(df)
df = set_subgroups(df, bonus_coeff)

In [ ]:
print("РАЗБИВКА ВОДИТЕЛЕЙ ПО ГРУППАМ:")

print("\ntotal drivers:", df.shape[0])
for cohort in range (1, cohorts + 1):
    
    cohort_n = df['cohort'] == cohort
    group_n = df['group'] == group
    
    print(" cohort", cohort, ":", df[cohort_n].shape[0])
    for group in set(df['group']):
        print("  ", group, ":", df[cohort_n & group_n].shape[0])

РАЗБИВКА ВОДИТЕЛЕЙ ПО ГРУППАМ:

total drivers: 5926


NameError: ignored

## Assign bonuses to drivers

In [ ]:
# assign target rides and bonuses for each drivers' id
df['target_rides'] = df['cohort'].map(rides_current)
df['bonus'] = df['cohort'].map(driver_bonus)

df['bonus_adj'] = df['bonus'] * df['bonus_coeff']

## Сохранаяем расчеты в файл

file name format is "bonus_table_day_month_hour_min"<br>
'id' - driver id<br>
'bonus' - initial bonus<br>
'bonus_coeff' - reducing coeff for bonus<br>
'bonus_adj' - final bonus<br>
'target_rides' - target_rides rides quantity<br>

In [ ]:
features = ['id', 'cohort', 'target_rides', 'group', 'bonus', "bonus_coeff", 'bonus_adj']
file_name = "bonus_table_" + datetime.today().strftime('%d%m_%H%M') + '.csv'
df[features].to_csv(file_name, index=False)
print("all done.")

all done.
